<a href="https://colab.research.google.com/github/danieltalon/trabalho_RAG/blob/main/1_ler_json_com_perguntas_classificar_gerar_novo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
'''
Resumo: Esse script le um arquivo JSON com perguntas de vestibular de Medicina,
envia para uma IA classificar as perguntas dentro de áreas da medicina,
e por fim, gera um novo JSON com as perguntas classificadas, para podermos
filtrar e agrupar por áreas médicas

Autor: Daniel Talon
Data: Abril 2025
'''

# --- PASSO 1: Instalar a biblioteca ---
%pip install -q -U google-generativeai

# --- PASSO 2: Importações ---
import json
import textwrap
import google.generativeai as genai
from google.colab import userdata
import time
import ast
from collections import Counter
import os
from google.api_core import exceptions as google_api_exceptions
import requests
import math

# ==============================================================================
# --- BLOCO DE CONFIGURAÇÃO ---
# ==============================================================================

# --- Parâmetros de Execução ---

# ANOS A PROCESSAR: Coloque os anos desejados em uma lista.
# Exemplo: [2024] para processar apenas 2024
# Exemplo: [2023, 2024] para processar 2023 e 2024
# Exemplo: [] para processar TODOS os anos (cuidado com a quota da API!)
TARGET_YEARS = [2024,2023,2022,2021] # <<< MODIFIQUE AQUI OS ANOS DESEJADOS

# MÁXIMO DE QUESTÕES A PROCESSAR: Limita o número total de questões enviadas à API.
# Útil para testes. Defina como None ou <= 0 para processar todas as questões filtradas.
MAX_QUESTIONS_TO_PROCESS = 700 # <<< MODIFIQUE AQUI PARA TESTAR (ex: 10, 50, ou None)

# TAMANHO DO LOTE: Questões por chamada de API.
# Valores menores (ex: 10-25) são mais seguros contra erros de contexto/parse da IA.
# Valores maiores reduzem o número de chamadas, mas aumentam o risco de erros.
BATCH_SIZE = 25 # <<< AJUSTE SE NECESSÁRIO

# PAUSA ENTRE LOTES: Tempo (segundos) de espera entre lotes. Aumente se receber erro 429.
# 5 segundos = ~12 chamadas/min; 10 segundos = ~6 chamadas/min
PAUSA_ENTRE_LOTES = 6 # <<< AJUSTE SE NECESSÁRIO (em segundos)

# MÁXIMO DE TENTATIVAS POR LOTE: Em caso de erro de conexão/API.
MAX_TENTATIVAS_LOTE = 3

# --- Configurações da API e Modelo ---
API_KEY_SECRET_NAME = 'GOOGLE_API_KEY' # Nome do segredo no Colab
MODEL_NAME = 'gemini-1.5-flash'       # Modelo Gemini a ser usado
OUTPUT_FILE_NAME_SUFFIX = '_classificadas_lote.json' # Sufixo para o nome do arquivo de saída

# Configurações de geração da IA
GENERATION_CONFIG = genai.GenerationConfig(
    temperature=0.2,
    top_p=0.9,
    top_k=40,
    # max_output_tokens=8192, # Descomente e ajuste se necessário para lotes maiores
    response_mime_type="application/json" # Solicita explicitamente saída JSON
)

# Configurações de segurança da IA
SAFETY_SETTINGS = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

# --- Caminhos dos Arquivos ---
INPUT_FILE_PATH = '/content/questoes-Revalida+Fuvest.json'
# Gera nome do arquivo de saída baseado nos anos ou 'todos' e no limite
years_str = "_".join(map(str, sorted(TARGET_YEARS))) if TARGET_YEARS else "todos_anos"
limit_str = f"_limit{MAX_QUESTIONS_TO_PROCESS}" if (MAX_QUESTIONS_TO_PROCESS is not None and MAX_QUESTIONS_TO_PROCESS > 0) else ""
OUTPUT_FILE_PATH = f'/content/questoes_{years_str}{limit_str}{OUTPUT_FILE_NAME_SUFFIX}'

# ==============================================================================
# --- FIM DO BLOCO DE CONFIGURAÇÃO ---
# ==============================================================================

# --- Configuração da API Key ---
try:
    GOOGLE_API_KEY = userdata.get(API_KEY_SECRET_NAME)
    if not GOOGLE_API_KEY:
        raise ValueError("API Key não encontrada ou vazia.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key carregada com sucesso.")
except Exception as e:
    print(f"Erro ao carregar a API Key '{API_KEY_SECRET_NAME}': {e}")
    print("Por favor, certifique-se de que adicionou a chave aos Secrets do Colab e ativou o 'Notebook access'.")
    raise SystemExit("API Key não configurada.")

# --- Configuração do Modelo ---
try:
    model = genai.GenerativeModel(MODEL_NAME)
    print(f"Modelo Gemini '{MODEL_NAME}' configurado.")
except Exception as e:
    print(f"Erro ao configurar o modelo Gemini '{MODEL_NAME}': {e}")
    raise SystemExit("Falha na configuração do modelo.")

# --- Leitura do JSON de Entrada ---
all_data = []
print(f"\nTentando ler o arquivo de entrada: {INPUT_FILE_PATH}")
if not os.path.exists(INPUT_FILE_PATH):
    print(f"Erro: Arquivo '{INPUT_FILE_PATH}' não encontrado.")
    print("Certifique-se de que o arquivo foi carregado corretamente no Colab.")
else:
    try:
        with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as f:
            all_data = json.load(f)
        print(f"Arquivo '{INPUT_FILE_PATH}' lido com sucesso. {len(all_data)} questões encontradas no total.")
    except json.JSONDecodeError:
        print(f"Erro: O arquivo '{INPUT_FILE_PATH}' não é um JSON válido.")
    except Exception as e:
        print(f"Erro inesperado ao ler o arquivo: {e}")

# --- Filtragem e Limitação dos Dados ---
data_to_process = []
if all_data:
    # Filtra por ano
    if TARGET_YEARS:
        data_filtered_by_year = [q for q in all_data if q.get('prova') in TARGET_YEARS]
        years_str_msg = ", ".join(map(str, TARGET_YEARS))
        print(f"\nFiltrado para o(s) ano(s): {years_str_msg}. {len(data_filtered_by_year)} questões encontradas.")
    else:
        data_filtered_by_year = all_data
        print("\nNenhum ano específico selecionado. Considerando todas as questões.")

    # Aplica o limite máximo
    if MAX_QUESTIONS_TO_PROCESS is not None and MAX_QUESTIONS_TO_PROCESS > 0:
        if len(data_filtered_by_year) > MAX_QUESTIONS_TO_PROCESS:
            print(f"Limitando o processamento às primeiras {MAX_QUESTIONS_TO_PROCESS} questões filtradas.")
            data_to_process = data_filtered_by_year[:MAX_QUESTIONS_TO_PROCESS]
        else:
            print(f"Número de questões filtradas ({len(data_filtered_by_year)}) é menor ou igual ao limite ({MAX_QUESTIONS_TO_PROCESS}). Processando todas as filtradas.")
            data_to_process = data_filtered_by_year
    else:
        print("Nenhum limite máximo de questões definido. Processando todas as questões filtradas.")
        data_to_process = data_filtered_by_year

    if not data_to_process:
         print("Nenhuma questão selecionada para processamento após filtros/limites.")

else:
    print("Nenhum dado carregado do arquivo JSON.")

# --- Processamento em Lotes ---
if data_to_process:
    print(f"\nIniciando classificação de {len(data_to_process)} questões em lotes...")
    area_counter = Counter()
    total_questoes_a_processar = len(data_to_process)
    questoes_processadas_sucesso = 0
    erros_api_lote = 0
    erros_parse_lote = 0
    questoes_nao_retornadas_no_lote = 0

    num_lotes = math.ceil(total_questoes_a_processar / BATCH_SIZE)
    print(f"Processando em {num_lotes} lotes de até {BATCH_SIZE} questões cada.")

    for i in range(0, total_questoes_a_processar, BATCH_SIZE):
        lote_questoes = data_to_process[i : i + BATCH_SIZE] # Pega o slice correto dos dados a processar
        lote_numero = (i // BATCH_SIZE) + 1
        print(f"\nProcessando Lote {lote_numero}/{num_lotes} (Questões {i+1} a {min(i+BATCH_SIZE, total_questoes_a_processar)})...")

        # Construir o prompt para o lote
        prompt_lote = """
        Analise os enunciados das questões médicas abaixo, identificadas por 'ID: [Prova]-[Numero]'.
        Para CADA questão, classifique-a em uma ou mais áreas principais da medicina (exemplos: Pediatria, Obstetrícia, Ginecologia, Cardiologia, Cirurgia Geral, Clínica Médica, Preventiva/Saúde Coletiva, Infectologia, Psiquiatria, Dermatologia, Neurologia, Ortopedia, Oftalmologia, Otorrinolaringologia, Nefrologia, Endocrinologia, Gastroenterologia, Pneumologia, Hematologia, Oncologia, Ética Médica, Medicina Legal, Saúde do Trabalhador, Geriatria, Urgência/Emergência).
        Se uma questão abordar múltiplos temas, liste todas as áreas relevantes. Se for muito geral, use 'Clínica Médica Geral'.

        Retorne sua resposta ESTRITAMENTE como um único dicionário JSON válido, onde as chaves são os 'ID' das questões (no formato 'Prova-Numero') e os valores são listas de strings contendo as áreas médicas identificadas para aquela questão específica. Garanta que o JSON esteja completo e bem formatado.

        Exemplo de formato de saída JSON esperado:
        {
          "2011-1": ["Endocrinologia", "Gastroenterologia", "Clínica Médica Geral"],
          "2011-2": ["Pediatria", "Hematologia", "Oncologia", "Infectologia"]
        }

        Questões para classificar:
        """
        ids_no_lote = []
        # Itera sobre o lote atual para construir o prompt
        for q_idx_lote, questao_lote in enumerate(lote_questoes):
            # Usa o índice global (i + q_idx_lote) para gerar ID único se 'numero' faltar
            prova = questao_lote.get('prova', 'PNA')
            numero = questao_lote.get('numero', f'idx{i+q_idx_lote+1}')
            enunciado = questao_lote.get('enunciado', '')
            q_id = f"{prova}-{numero}"
            ids_no_lote.append(q_id)
            prompt_lote += f"\n\nID: {q_id}\nEnunciado: {enunciado}\n---"

        prompt_lote += "\n\nDicionário JSON com as classificações:"

        # Chamar a API para o lote com retentativas
        tentativa_atual = 0
        sucesso_lote = False
        response_lote = None

        while tentativa_atual < MAX_TENTATIVAS_LOTE and not sucesso_lote:
            tentativa_atual += 1
            try:
                response_lote = model.generate_content(
                    prompt_lote,
                    generation_config=GENERATION_CONFIG,
                    safety_settings=SAFETY_SETTINGS
                )
                sucesso_lote = True

            except google_api_exceptions.ResourceExhausted as e:
                print(f"  [ERRO API Lote {lote_numero} - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_LOTE}: Quota Excedida (429). Esperando {PAUSA_ENTRE_LOTES * 5}s...]")
                time.sleep(PAUSA_ENTRE_LOTES * 5)
            except (requests.exceptions.ConnectionError, google_api_exceptions.ServiceUnavailable, ConnectionResetError, google_api_exceptions.InternalServerError) as e:
                print(f"  [ERRO API Lote {lote_numero} - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_LOTE}: Erro de conexão/servidor ({type(e).__name__}). Esperando {PAUSA_ENTRE_LOTES}s...]")
                time.sleep(PAUSA_ENTRE_LOTES)
            except google_api_exceptions.InvalidArgument as e:
                 print(f"  [ERRO API Lote {lote_numero} - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_LOTE}: Argumento Inválido (400). Pode ser limite de contexto. Erro: {e}]")
                 sucesso_lote = True # Sai do loop, erro provavelmente não recuperável
                 response_lote = None # Garante que não tentará processar resposta inválida
            except Exception as e:
                print(f"  [ERRO GERAL INESPERADO Lote {lote_numero} - Tentativa {tentativa_atual}/{MAX_TENTATIVAS_LOTE}: {type(e).__name__} - {e}]")
                sucesso_lote = True # Sai do loop
                response_lote = None # Garante que não tentará processar resposta inválida
                time.sleep(PAUSA_ENTRE_LOTES * 2)

        # Processar a resposta do lote
        if sucesso_lote and response_lote:
            classificacoes_lote_dict = {}
            texto_resposta_limpo = ""
            try:
                if response_lote.parts:
                    texto_resposta = response_lote.text.strip()
                    # Limpeza básica de markdown
                    if texto_resposta.startswith("```json"):
                        texto_resposta = texto_resposta[len("```json"):].strip()
                    elif texto_resposta.startswith("```"):
                        texto_resposta = texto_resposta[len("```"):].strip()
                    if texto_resposta.endswith("```"):
                        texto_resposta = texto_resposta[:-len("```")].strip()
                    texto_resposta_limpo = texto_resposta

                    classificacoes_lote_dict = json.loads(texto_resposta_limpo)

                    if not isinstance(classificacoes_lote_dict, dict):
                         raise TypeError("Resposta da IA não é um dicionário JSON.")

                    # Atualizar as questões no 'data_to_process'
                    # Itera sobre o lote original para atualizar os dicionários corretos
                    for q_idx_lote, questao_original in enumerate(lote_questoes):
                        prova = questao_original.get('prova', 'PNA')
                        numero = questao_original.get('numero', f'idx{i+q_idx_lote+1}')
                        q_id = f"{prova}-{numero}"

                        if q_id in classificacoes_lote_dict:
                            areas = classificacoes_lote_dict[q_id]
                            if isinstance(areas, list) and all(isinstance(item, str) for item in areas):
                                # Atualiza o dicionário original na lista data_to_process
                                questao_original['areas_medicas'] = [area.strip() for area in areas if area.strip()] or ['Não Classificado - Lista Vazia']
                                area_counter.update(questao_original['areas_medicas'])
                                questoes_processadas_sucesso += 1
                            else:
                                print(f"  [AVISO Lote {lote_numero}: Formato inválido para ID {q_id}. Valor: {areas}]")
                                questao_original['areas_medicas'] = ['Não Classificado - Formato Inválido']
                                erros_parse_lote += 1
                        else:
                            print(f"  [AVISO Lote {lote_numero}: ID {q_id} não encontrado na resposta da IA.]")
                            questao_original['areas_medicas'] = ['Não Classificado - ID Ausente']
                            questoes_nao_retornadas_no_lote += 1
                else:
                    # Tratamento para resposta bloqueada ou vazia
                    block_reason = response_lote.prompt_feedback.block_reason if response_lote.prompt_feedback else "Razão desconhecida"
                    print(f"  [AVISO Lote {lote_numero}: Resposta vazia ou bloqueada pela IA. Razão: {block_reason}]")
                    for q in lote_questoes: q['areas_medicas'] = ['Não Classificado - Lote Bloqueado']
                    erros_api_lote += len(lote_questoes)

            except (json.JSONDecodeError, TypeError) as parse_error:
                print(f"  [ERRO ao parsear JSON do Lote {lote_numero}: {parse_error}. Resposta recebida (após limpeza): '{texto_resposta_limpo}']")
                for q in lote_questoes: q['areas_medicas'] = ['Não Classificado - Erro Parse Lote']
                erros_parse_lote += len(lote_questoes)
            except Exception as e:
                print(f"  [ERRO inesperado ao processar resposta do Lote {lote_numero}: {e}]")
                for q in lote_questoes: q['areas_medicas'] = ['Não Classificado - Erro Inesperado Lote']
                erros_parse_lote += len(lote_questoes)

        elif not sucesso_lote: # Se falhou após todas as tentativas
            print(f"  [ERRO API Lote {lote_numero}: Falha ao obter resposta após {MAX_TENTATIVAS_LOTE} tentativas.]")
            for q in lote_questoes: q['areas_medicas'] = ['Erro API - Lote Falhou']
            erros_api_lote += len(lote_questoes)

        # Pausa entre lotes, independentemente do sucesso ou falha do lote
        print(f"  Lote {lote_numero} concluído. Pausando por {PAUSA_ENTRE_LOTES} segundos...")
        time.sleep(PAUSA_ENTRE_LOTES)

    # --- PASSO 5: Salvar o Novo JSON (APENAS os dados processados) ---
    print(f"\nSalvando os resultados classificados em: {OUTPUT_FILE_PATH}")
    try:
        # Salva a lista 'data_to_process' que contém apenas as questões que foram selecionadas e processadas
        with open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as f:
            json.dump(data_to_process, f, ensure_ascii=False, indent=4)
        print("Arquivo salvo com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar o arquivo JSON de saída: {e}")

    # --- PASSO 6: Gerar Resumo ---
    print("\n--- Resumo da Classificação por Área ---")
    years_str_msg = ", ".join(map(str, TARGET_YEARS)) if TARGET_YEARS else "Todos os anos"
    limit_msg = f"(limitado a {MAX_QUESTIONS_TO_PROCESS})" if (MAX_QUESTIONS_TO_PROCESS is not None and MAX_QUESTIONS_TO_PROCESS > 0) else ""
    print(f"Anos processados: {years_str_msg} {limit_msg}")
    print(f"Total de questões processadas: {total_questoes_a_processar}")
    print(f"Questões classificadas com sucesso: {questoes_processadas_sucesso}")
    print(f"Questões com erro na chamada da API (Quota/Conexão/Outros): {erros_api_lote}")
    print(f"Questões com erro no parse da resposta da IA: {erros_parse_lote}")
    print(f"Questões não retornadas pela IA dentro de um lote: {questoes_nao_retornadas_no_lote}")
    print("-" * 20)
    if area_counter:
        # Exclui categorias de erro/não classificação do resumo principal
        areas_validas = {k: v for k, v in area_counter.items() if not k.startswith('Erro') and not k.startswith('Não Classificado')}
        if areas_validas:
             print("Contagem por área (excluindo erros/não classificados):")
             for area, contagem in Counter(areas_validas).most_common():
                 print(f"{area}: {contagem} questões")
        else:
             print("Nenhuma área válida foi classificada.")
    else:
        print("Nenhuma classificação foi realizada com sucesso.")

elif not all_data:
     print("\nNenhum dado foi carregado do arquivo JSON. Classificação e salvamento cancelados.")
elif not data_to_process:
     print("\nNenhuma questão selecionada para processamento após filtros/limites.")

API Key carregada com sucesso.
Modelo Gemini 'gemini-1.5-flash' configurado.

Tentando ler o arquivo de entrada: /content/questoes-Revalida+Fuvest.json
Arquivo '/content/questoes-Revalida+Fuvest.json' lido com sucesso. 1301 questões encontradas no total.

Filtrado para o(s) ano(s): 2024, 2023, 2022, 2021. 670 questões encontradas.
Número de questões filtradas (670) é menor ou igual ao limite (700). Processando todas as filtradas.

Iniciando classificação de 670 questões em lotes...
Processando em 27 lotes de até 25 questões cada.

Processando Lote 1/27 (Questões 1 a 25)...
  Lote 1 concluído. Pausando por 6 segundos...

Processando Lote 2/27 (Questões 26 a 50)...
  Lote 2 concluído. Pausando por 6 segundos...

Processando Lote 3/27 (Questões 51 a 75)...
  Lote 3 concluído. Pausando por 6 segundos...

Processando Lote 4/27 (Questões 76 a 100)...
  Lote 4 concluído. Pausando por 6 segundos...

Processando Lote 5/27 (Questões 101 a 125)...
  Lote 5 concluído. Pausando por 6 segundos...

P